# Collaborative Filtering

Kolaborativno filtriranje je tehnika kojom se ocjene filmova predvidjaju na osnovu slicnosti medju korisnicima, odnosno medju filmovima. Preporucuju se oni filmovi za koje je predvidjena ocjena najvisa.

U slucaju problema $cold$ $start$ sistemi zastovani na sadrzaju ($content-based$) su korisni, kao i kada informacije o korisnicima i proizvodima nisu dostupne. Medjutim, u poredjenju sa njima, CF pristup je siri i fleksibilniji za identifikaciju slicnih proizvoda.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

In [2]:
rating_df=pd.read_csv('dataset/ratings.csv')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies_df=pd.read_csv('dataset/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating_df=rating_df[:2000] # zbog brzeg pokretanja 

In [5]:
ocjene=np.linspace(rating_df.rating.min(), rating_df.rating.max(), 10)
print('Ocjene filmova:')
for o in ocjene: # moguce ocjene su 0.5, 1, 1.5, ..., 5
    print(o)

Ocjene filmova:
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0


In [6]:
n_users = rating_df.userId.unique().shape[0]
n_movies = rating_df.movieId.unique().shape[0]
print ('Ukupno: ' + str(n_users) + ' korisnika, ' + str(n_movies) + ' filma')

Ukupno: 18 korisnika, 1206 filma


In [7]:
df_pivot=rating_df.pivot_table(values='rating', index='userId', columns='movieId').fillna(0)

In [8]:
ratings=df_pivot.as_matrix()

In [9]:
ratings # matrica koja sadrzi ocjene koje je i. korisnik dao j. filmu
# 0 ako korisnik nije ocjenio film

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [3.5, 3. , 0. , ..., 0. , 0. , 0. ]])

In [10]:
def sparsity(ratings): # prorjedjenost matrice
    sparsity = float(len(ratings.nonzero()[0]))
    sparsity /= (ratings.shape[0] * ratings.shape[1])
    sparsity *= 100
    return 100-sparsity

In [11]:
sparsity(ratings)

90.78680670720472

In [12]:
print ('Sparsity: {}%'.format(round(sparsity(ratings), 2)))

Sparsity: 90.79%


### Podjela na skup za obucavanje i testiranje

In [13]:
def my_train_test_split(ratings, test_size): # skupovi za obucavanje i testiranje su istog oblika, ali su vrijednsti 
    # raspodijeljene: test_size ocjena ce biti u skupu za testiranje
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    rows = (ratings != 0).sum(1)
    
    for user in range(ratings.shape[0]):
        count_per_user=int(np.ceil(test_size*rows[user]))
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], # 0 jer je nonzero primjenjeno na vektor
                                        size=count_per_user,  # indexi na kojima nisu 0
                                        replace=False)
        # u skup za obucavanje se upisuju 0 na izabranim pozicijama, a u skup za testiranje se upisuju elementi 
        train[user, test_ratings] = 0
        test[user, test_ratings] = ratings[user, test_ratings] 
        
    # provjera da li su poznate ocjene ili u jednom ili u drugom skupu
    assert(np.all((train * test) == 0)) 
    return train, test

In [14]:
train, test = my_train_test_split(ratings, 0.2)

In [15]:
sparsity(train)

92.66169154228855

In [16]:
train.shape # skupovi za obucavanje i testiranje su istih dimenzija, s tim sto im je proredjenost drugacija

(18, 1206)

In [17]:
test.shape

(18, 1206)

In [18]:
sparsity(test)

98.12511516491615

# Item-based Collaborative Filtering

In [19]:
def movies_similarity(ratings, epsilon=1e-9):
    # dodaje se epsilon -> mala vrijednost u slucaju da je slicnost filmova 0
    return cosine_similarity(ratings.T)+epsilon

In [20]:
m_sim=movies_similarity(train)

In [21]:
def predict_movies_top_k(ratings, movies_similarity, k=10):
    pred = np.zeros(ratings.shape)
    for j in range(ratings.shape[1]):
        top_k_items=[np.argsort(movies_similarity)[j][::-1][1:k+1]]
       
        for i in range(ratings.shape[0]):
            pred[i, j] = movies_similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
            pred[i, j] /= np.sum(np.abs(movies_similarity[j, :][top_k_items]))        
    
    return pred

In [ ]:
m_pred=predict_movies_top_k(train, m_sim)

# User-based Collaborative Filtering

In [ ]:
def users_similarity(ratings, epsilon=1e-9):
    return cosine_similarity(ratings)+epsilon

In [ ]:
u_sim=users_similarity(train)
u_sim.shape

In [ ]:
def predict_users_top_k(ratings, users_similarity, k=10):
    pred = np.zeros(ratings.shape)

    for i in range(ratings.shape[0]):
        top_k_users=[np.argsort(users_similarity[:,i])[::-1][1:k]]
        #print(top_k_users)
        for j in range(ratings.shape[1]):
            pred[i,j]=ratings[:, j][top_k_users].mean()
    
    return pred

In [ ]:
u_pred=predict_users_top_k(train, u_sim)

### Validacija

In [ ]:
def get_rmse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return np.sqrt(mean_squared_error(pred, actual))

In [ ]:
def get_mae(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(pred, actual)

In [ ]:
print ('Item-based CF RMSE train: {}'.format(get_rmse(m_pred, train)))
print ('Item-based CF RMSE test: {}'.format(get_rmse(m_pred, test)))

print ('Item-based CF MAE train: {}'.format(get_mae(m_pred, train)))
print ('Item-based CF MAE test: {}'.format(get_mae(m_pred, test)))

In [ ]:
print ('User-based CF RMSE train: {}'.format(get_rmse(u_pred, train)))
print ('User-based CF RMSE test: {}'.format(get_rmse(u_pred, test)))

print ('User-based CF MAE train: {}'.format(get_mae(u_pred, train)))
print ('User-based CF MAE test: {}'.format(get_mae(u_pred, test)))

### Izbor vrijednosti za $k$ najblizih susjeda

In [ ]:
k_array = [5, 10, 15, 20]

u_train_mae = []
u_test_mae = []
m_test_mae = []
m_train_mae = []

for k in k_array:
    u_pred = predict_users_top_k(train, u_sim, k=k)
    m_pred = predict_movies_top_k(train, m_sim, k=k)
    
    u_train_mae += [get_mae(u_pred, train)]
    u_test_mae += [get_mae(u_pred, test)]
    
    m_train_mae += [get_mae(m_pred, train)]
    m_test_mae += [get_mae(m_pred, test)] 

In [ ]:
plt.plot(k_array, u_train_mae, label='User-based train')
plt.plot(k_array, u_test_mae, label='User-based test')
plt.plot(k_array, m_train_mae, label='Item-based train')
plt.plot(k_array, m_test_mae, label='Item-based test')
plt.legend(loc='best')
plt.xlabel('k', fontsize=10)
plt.ylabel('MSE', fontsize=10)

# Singular Value Decomposition

In [ ]:
U, sigma, Vt = svds(train, k = 3) # k singularnih vrijednosti i vektora
sigma=np.diag(sigma)

### Predvidjanje ocjena na osnovu dekomponovane matrice

In [ ]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [ ]:
print ('SVD CF MAE train: {}'.format(get_mae(predicted_ratings, train)))
print ('SVD CF MAE test: {}'.format(get_mae(predicted_ratings, test)))

In [ ]:
k_array = [1, 5] # broj singularnih vrijednosti i vektora

train_mae = []
test_mae = []

for k in k_array:
    
    U, sigma, Vt = svds(train, k = k)
    sigma=np.diag(sigma)
    
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    
    train_mae += [get_mae(all_user_predicted_ratings, train)]
    test_mae += [get_mae(all_user_predicted_ratings, test)]


In [ ]:
plt.plot(k_array, train_mae, label='SVD MAE train', linewidth=2)
plt.plot(k_array, test_mae, label='SVD MAE test', linewidth=2)
plt.legend(loc='best', fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel('k', fontsize=10)
plt.ylabel('MSE', fontsize=10)

### Preporuka filmova

In [ ]:
preds_df = pd.DataFrame(predicted_ratings, columns = df_pivot.columns)
preds_df.shape

In [ ]:
def recommend_movies(predictions_df, userId, movies_df, original_ratings_df, num_recommendations=5):
    
    # sortiranje predvidjenih ocjena filmova 
    user_row_number = userId - 1 # indeksiranje userId pocinje od 1
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # uzimanje podataka o korisniku iz polazne podataka
    user_data = original_ratings_df[original_ratings_df.userId == (userId)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))

    print ('Broj filmova koje je korisnik {0} vec ocjenio: {1}'.format(userId, user_full.shape[0]))
    print ('Broj neocjenjenih filmova za koje su predvidjene najvise ocjene: {0}'.format(num_recommendations))
    
    # preporuka neocjenjenih filmova za koje su predvidjene najvise ocjene
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds_df, 1, movies_df, rating_df, 5)

In [ ]:
already_rated.head()

In [ ]:
predictions

# Implementacija metoda Stohasticki gradijentni spust

Metod latentnih faktora za CF koristi postojece ocjene filmova kako bi naucio istovremeno latentne karakterisnike i korisnika i filmova. Ovo je dosta mocniji pristup u poredjenju sa metodom najblizih susjeda.

In [ ]:
# Root Mean Squared Error
def rmse_sgd(train, Q, P):
    I = train != 0  # indikatorska funkcija o ocjenjenim filmovima
    ME = I * (train - np.dot(P, Q.T))  # greska izmedju stvarnih i predvidjenih vrijednosti
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I)) 

In [ ]:
# parametri i inicijalizacija latentnih faktora
f = 5  # broj parova latentnih faktora
lmbda = 0.5 # Regularisation strength
gamma=0.01  # Learning rate
n_epochs = 10  # Number of loops through training data
P = np.sqrt(train.max())* np.random.rand(n_users, f) # Latentni faktori za korisnika
Q = np.sqrt(train.max())* np.random.rand(n_movies, f) # Latentni faktori za filmove

In [ ]:
# SGD
train_errors_per_epochs = []
test_errors_per_epochs = []
users,items = train.nonzero()      
for epoch in range(n_epochs):
    for u, i in zip(users,items):
        e = train[u, i] - np.dot(P[u, :], Q[i, :].T)  # greska kao razlika poznate i pretpostavljene ocjene
        P[u, :] += gamma * ( e * Q[i, :] - lmbda * P[u, :]) # azuriranje nove vrijednosti koja opisuje koristika
        Q[i, :] += gamma * ( e * P[u, :] - lmbda * Q[i, :])  # azuriranje nove vrijednosti koja opisuje film
    train_errors_per_epochs.append(rmse_sgd(train,Q,P)) # Training RMSE for this pass
    test_errors_per_epochs.append(rmse_sgd(test,Q,P)) # Test RMSE for this pass

In [ ]:
# provjeravanje performansi grafickim prikazom greske na skupu za obucavanje i testiranje
plt.plot(train_errors_per_epochs, color="g", label='Training RMSE')
plt.plot(test_errors_per_epochs, color="r", label='Test RMSE')
plt.xlabel('Number of Epochs')
plt.ylabel('RMSE')
plt.title('Error During Stochastic GD')
plt.legend()
# Tokom procesa obucavanja, greska na skupu za obucavanje ima trend opadanja

In [ ]:
predicted_train=np.dot(P, Q.T)
predicted_train

In [ ]:
print('SVD CF MAE train: {}'.format(get_mae(predicted_train, train)))
print('SVD CF RMSE train: {}'.format(get_rmse(predicted_train, train)))

### Validacija na test skupu

In [ ]:
f = 5  # broj parova latentnih faktora
lmbda = 0.5 # Regularisation strength
gamma=0.01  # Learning rate
n_epochs = 10  
P = np.sqrt(test.max())* np.random.rand(n_users, f) # Latentni faktori za korisnika
Q = np.sqrt(test.max())* np.random.rand(n_movies, f) 

In [ ]:
# SGD na test skupu
users,items = test.nonzero()      
for epoch in range(n_epochs):
    for u, i in zip(users,items):
        e = test[u, i] - np.dot(P[u, :], Q[i, :].T)  
        P[u, :] += gamma * ( e * Q[i, :] - lmbda * P[u, :]) 
        Q[i, :] += gamma * ( e * P[u, :] - lmbda * Q[i, :]) 

In [ ]:
predicted_test=np.dot(P, Q.T)
predicted_test

In [ ]:
print('SVD CF MAE test: {}'.format(get_mae(predicted_test, test)))
print('SVD CF RMSE test: {}'.format(get_rmse(predicted_test, test)))

In [ ]:
# Nad svim poznatim vrijednostima
f = 5 
lmbda = 0.5 # Regularisation strength
gamma=0.01  # Learning rate
n_epochs = 10  
P = np.sqrt(ratings.max())* np.random.rand(n_users, f) 
Q = np.sqrt(ratings.max())* np.random.rand(n_movies, f)

In [ ]:
users,items = train.nonzero()      
for epoch in range(n_epochs):
    for u, i in zip(users,items):
        e = ratings[u, i] - np.dot(P[u, :], Q[i, :].T) 
        P[u, :] += gamma * ( e * Q[i, :] - lmbda * P[u, :]) 
        Q[i, :] += gamma * ( e * P[u, :] - lmbda * Q[i, :])

In [ ]:
predicted=np.dot(P, Q.T)
predicted

In [ ]:
print('SVD CF MAE: {}'.format(get_mae(predicted, ratings)))
print('SVD CF RMSE: {}'.format(get_rmse(predicted, ratings)))